In [1]:
# library imports
import csv
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
# import NDP file (~3GB of plain text)
ndp_file = "227-www-ndp-ca.txt"

In [3]:
# load the NDP plain text as a dataframe
df = pd.read_csv(ndp_file, sep = ",", usecols=[0,1,2,3], header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df = df.sort_values(0)

In [4]:
# let's focus just on the homepage
# find a homepage to focus on for the diffs
homepages = df.loc[df[2].isin(["http://www.ndp.ca/","https://www.ndp.ca/"])]
print(len(homepages))
homepage_text = homepages[3]

101


In [5]:
# let's begin by comparing texts
# we can load the SequenceMatcher library

from difflib import SequenceMatcher
lists = [ [] for _ in range(5) ]
labels = []


In [6]:
dates = homepages[0].tolist()
newdates = [x[1:] for x in dates]
lists[0] = newdates
labels.append('dates')

In [7]:
# byte-wise metric
# compares two pages character by character
# returns 1 if there is any change at all between two pages

bytewise = []
for y in range(len(homepages)-1):
    first_page = homepage_text.iloc[y]
    second_page = homepage_text.iloc[y+1]
    if len(first_page) == len(second_page):
        for k in range(len(first_page)):
            if first_page[k] != second_page[k]:
                bytewise.append(1)
                break   
            elif (k == len(first_page) - 1) and (first_page[k] == second_page[k]):
                bytewise.append(0)
    else:
        bytewise.append(1)

# output is the list of metric values: either 0 (no change) or 1 (any change)
# print(bytewise)
lists[1] = bytewise
labels.append('byte-wise')

In [8]:
# TF.IDF metric
# compares the occurrence of chosen words in each page
# generates vectors weighted by such occurrences and calculates cosine distance between them

import re
import random
import numpy as np


tfidf = []
for y in range(len(homepages)-1):
    
    first_page = re.sub("[^\w]", " ",  "".join(homepage_text.iloc[y])).split()
    second_page = re.sub("[^\w]", " ",  "".join(homepage_text.iloc[y+1])).split()

    p1 = [x for x in first_page if re.match("\w+",x)]
    p2 = [s.lower() for s in p1]
    p3 = [word for word in p2 if word not in stop_words]
    
    first_page_unique = np.unique(np.asarray(p3))
    
    v1 = []
    v2 = []
     
#     random_words = []
#     for i in range(10):
#         rand = random.choice(p3)
#         print(rand)
#         while len(rand) <= 4 or (rand in random_words):
#             rand = random.choice(p3)    
#         random_words.append(rand)

#     for j in range(len(random_words)):
#         v1.append(first_page.count(random_words[j]))
#         v2.append(second_page.count(random_words[j]))
        
    for j in range(len(first_page_unique)):
        v1.append(first_page.count(first_page_unique[j]))
        v2.append(second_page.count(first_page_unique[j]))
    
    if np.linalg.norm(v2) == 0:
        D = 1
    else:
        D = 1 - np.inner(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    tfidf.append(D)

# output is the list of metric values between 0 (very similar) and 1 (very different)
# print(tfidf)
lists[2] = tfidf
labels.append('TF.IDF')

In [ ]:
# edit distances metric
# number of edits that are required to transform one sentence into another.
# we normalise this number by the total number of characters in each of the two pages 
# ie deleting all of page 1 and adding all of page 2

edit_distances = []
for y in range(len(homepages)-1):
    first_page = "".join(homepage_text.iloc[y]) # convert webpage text into one long string
    second_page = "".join(homepage_text.iloc[y+1]) 
    len_first = len(first_page)
    len_second = len(second_page)
    
    distance = nltk.edit_distance(first_page, second_page, transpositions=False)
    edit_distances.append(distance / max(len_first,len_second))

# output is the list of metric values between 0 (very similar) and 1 (very different)
# print(edit_distances)
lists[3] = edit_distances
labels.append('edit distances')

In [ ]:
# word distance metric 
# calculates how many of the words on a page have changed
# split option allows us to consider if splitting the strings into words makes a difference

word_distance = []
word_distance_split = []
for y in range (0,len(homepages)-1):
    first_page = homepages.iloc[y]
    second_page = homepages.iloc[y+1]
    seq = SequenceMatcher(None, first_page[3],second_page[3])
    seq_split = SequenceMatcher(None, first_page[3].split(),second_page[3].split())
    distance = 1-seq.ratio()
    distance_split = 1-seq_split.ratio()
    word_distance.append(distance)
    word_distance_split.append(distance_split)
    
# output is the list of metric values between 0 (very similar) and 1 (very different)
# print(word_distance)
# print(word_distance_split)
lists[4] = word_distance
#lists[5] = word_distance_split
labels.append('word distances')
#labels.append('word distances split')

In [ ]:
times = homepages[0].unique()
N = 10

plt.figure(figsize = (15,5), dpi = 80, facecolor = 'w', edgecolor = 'k')
for j in range(1, len(lists)):
    my_xticks = []
    for i in range (0,len(times)-1,N):
        my_xticks.append(str(times[i][5:7] + "/" + times[i][3:5]));
    plt.plot(lists[j], label = labels[j]);
    plt.xticks(range(0,len(times)-1,N), my_xticks);
    plt.legend(loc = "upper left", fontsize = 14)

In [67]:
import csv

with open("text_metrics.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(list(map(list, zip(*lists))))

In [ ]:
## Trying to go one page deeper than homepage -- having problems with dataframes
# find all links that begin with "http://www.ndp.ca/"
subset_mask = df[2].str[:18] == "http://www.ndp.ca/"
subset = df.loc[subset_mask]
# df[2] = df[2].str[:18] # not sure which number
# print(df[2].unique())

# reduce data to those within 2 pages of homepage
nextpage_mask = subset[2].str.len() > 18
slash_mask = (subset[2].str.count('/') <= 4)
subset = subset.loc[nextpage_mask]
subset = subset.loc[slash_mask]

# add slashes to data
for i in range(len(subset[2])):
    string = subset.loc[subset.index[i], 2]
    print(string)
    final_char = string[len(string)-1]
    if final_char != '/':
        subset[2][i] = string + '/'
    else:
        subset[2][i] = string

slash_mask2 = (subset[2].str.count('/') == 4)
subset = subset.loc[slash_mask2]
print(subset)

#lngth = []
#for i in range(len(df[2])):
#    locations = [pos for pos, char in enumerate(df[2][i]) if char == '/']
#    lngth.append(len(locations))
# print(lngth)

#nextpages = df.loc[lngth == 3]

#nextpages = df.loc[len(locations) == 4]